In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import time

In [2]:
df = pd.read_csv('./events_250k.csv')

In [3]:
# change format to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], format = "%Y-%m-%d %H:%M:%S")
df.head()

,Unnamed: 0,event_id,device_id,timestamp,longitude,latitude
0,1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
1,2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
2,3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
3,4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66
4,6,7,5990807147117726237,2016-05-01 00:15:13,113.73,23.00


In [4]:
# sort timestamp value
df = df.sort_values('timestamp')
df.head()

,Unnamed: 0,event_id,device_id,timestamp,longitude,latitude
225486,333162,333163,8318950298181419630,2016-04-30 23:53:05,114.20,36.71
197659,291991,291992,8189881002794531237,2016-04-30 23:53:05,121.34,30.72
108726,160530,160531,-3879335903763469416,2016-04-30 23:53:59,122.13,40.25
47,56,57,8292388373526453818,2016-04-30 23:54:44,116.46,39.89
81223,120011,120012,2346793034696706898,2016-04-30 23:55:05,118.08,24.46


In [5]:
# reset index
df = df.reset_index(drop=True)
df

,Unnamed: 0,event_id,device_id,timestamp,longitude,latitude
0,333162,333163,8318950298181419630,2016-04-30 23:53:05,114.20,36.71
1,291991,291992,8189881002794531237,2016-04-30 23:53:05,121.34,30.72
2,160530,160531,-3879335903763469416,2016-04-30 23:53:59,122.13,40.25
3,56,57,8292388373526453818,2016-04-30 23:54:44,116.46,39.89
4,120011,120012,2346793034696706898,2016-04-30 23:55:05,118.08,24.46
...,...,...,...,...,...,...
249995,325068,325069,-3903059712853419884,2016-05-07 23:55:51,109.19,34.74
249996,63154,63155,3915082290673137129,2016-05-07 23:55:57,116.39,39.91
249997,266646,266647,-4360669416573647160,2016-05-07 23:55:58,110.52,24.20
249998,239697,239698,8371894969064070025,2016-05-07 23:57:02,114.14,30.49


In [6]:
# create cluster column
df['cluster'] = np.nan 
df.head()

,Unnamed: 0,event_id,device_id,timestamp,longitude,latitude,cluster
0,333162,333163,8318950298181419630,2016-04-30 23:53:05,114.20,36.71,NaN
1,291991,291992,8189881002794531237,2016-04-30 23:53:05,121.34,30.72,NaN
2,160530,160531,-3879335903763469416,2016-04-30 23:53:59,122.13,40.25,NaN
3,56,57,8292388373526453818,2016-04-30 23:54:44,116.46,39.89,NaN
4,120011,120012,2346793034696706898,2016-04-30 23:55:05,118.08,24.46,NaN


In [7]:
# test function isnull
print(pd.isnull(df['longitude'][10]))
print(pd.isnull(df['cluster'][10]))

False
True


In [8]:
# divide into clusters based on 3 conditions, priortize timestamp
# algorithm 1, presumably ineffective
no_group = 0
df['cluster'][0] = no_group 

for i in range(0, 250000):
    if pd.isnull(df['cluster'][i]) == False:
        pass
    else:
        df['cluster'][i] = no_group
        for j in range(i+1, 250000):
            if (df['device_id'][j] == df['device_id'][i]):
                # different in device
                pass
            else:
                if ((df['timestamp'][j] - df['timestamp'][i]).total_seconds() > 180):
                    # time difference > 180 secs
                    pass
                else:
                    if (math.sqrt(math.pow(df['longitude'][j] - df['longitude'][i],2) + math.pow(df['latitude'][j] - df['latitude'][i],2)) > 0.1):
                        # distance > 0.1
                        pass
                    else:
                        df['cluster'][j] = df['cluster'][i]
    no_group += 1

NameError: name 'df' is not defined

In [7]:
# divide into clusters based on 3 conditions, priortize timestamp
# algorithm 2, also presumably ineffective as 3 conditions have to be checked simutaneously without prioritization

no_group = 0
df['cluster'][0] = no_group 

for i in range(0, 250000):
    if pd.isnull(df['cluster'][i]) == False:
        pass
    else:
        df['cluster'][i] = no_group
        for j in range(i+1, 250000):
            if (df['device_id'][j] != df['device_id'][i]) and \
               ((df['timestamp'][j] - df['timestamp'][i]).total_seconds() <= 180) and \
               (math.sqrt(math.pow(df['longitude'][j] - df['longitude'][i],2) + math.pow(df['latitude'][j] - df['latitude'][i],2)) <= 0.1):
                df['cluster'][j] = df['cluster'][i]
            else:
                pass
    no_group += 1

NameError: name 'df' is not defined

In [8]:
# divide into clusters based on 3 conditions, priortize timestamp
# algorithm 3, presume correct and most effective
start = time.time()
no_group = 0
df['cluster'][0] = no_group 

for i in range(0, 250000):
    
    """ Code như cắn vào đít tao
    if pd.isnull(df['cluster'][i]) == False:
        pass
    else:
        df['cluster'][i] = no_group
        for j in range(i+1, 250000):
            if ((df['timestamp'][j] - df['timestamp'][i]).total_seconds() > 180):
                break
            else:
                if (df['device_id'][j] == df['device_id'][i]):
                    continue
                else:
                    if (math.sqrt(math.pow(df['longitude'][j] - df['longitude'][i],2) + math.pow(df['latitude'][j] - df['latitude'][i],2)) > 0.1):
                        continue
                    else:
                        df['cluster'][j] = df['cluster'][i]
    no_group += 1
    """
    
    if pd.isnull(df['cluster'][i]):
        df['cluster'][i] = no_group
        
        for j in range(i+1, 250000):
            
            if ((df['timestamp'][j] - df['timestamp'][i]).total_seconds() > 180):
                break
                
            if (df['device_id'][j] != df['device_id'][i]) \
            and (math.sqrt(math.pow(df['longitude'][j] - df['longitude'][i],2) \
            + math.pow(df['latitude'][j] - df['latitude'][i],2)) <= 0.1):
                df['cluster'][j] = df['cluster'][i]
                
    no_group += 1
    
total = time.time()-start
print(f'Time elapsed: {total}s')

c:\users\hp omen 15\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\hp omen 15\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\hp omen 15\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Time elapsed:
2655.8652288913727


In [ ]:
def dist(i,j):
    return (math.sqrt(math.pow(df['longitude'][j] - df['longitude'][i],2) \
            + math.pow(df['latitude'][j] - df['latitude'][i],2))

def time(i,j):
    return (df['timestamp'][j] - df['timestamp'][i]).total_seconds()

In [9]:
df['cluster'].describe()
df.to_csv('./finished_data.csv')
#print(time.time()-start)

count    250000.000000
mean     124993.887828
std       72168.639860
min           0.000000
25%       62493.000000
50%      124988.000000
75%      187491.250000
max      249999.000000
Name: cluster, dtype: float64